# 使用Geemap采样区域和点位的数据

In [9]:
import geopandas as gpd
import ee
import pandas as pd
from datetime import datetime, timedelta
# 统计降雨数据


# 初始化Google Earth Engine
ee.Initialize()

In [ ]:
# 读取本地Shapefile
shapefile_path = r"C:\Users\Runker\Desktop\gdv\qn.shp"
gdf = gpd.read_file(shapefile_path)

# 选择作为键的字段名，例如 'ID'
key_field1 = 'FXZQMC'
key_field2 = 'TXZQMC'

# 创建用于存储四至坐标的字典
bboxes_dict = {}

for _, row in gdf.iterrows():
    # 获取键字段的值
    key_value = row[key_field1]+row[key_field2]

    # 获取四至坐标
    bounds = row['geometry'].bounds
    bbox = {
        'MinX': bounds[0],
        'MinY': bounds[1],
        'MaxX': bounds[2],
        'MaxY': bounds[3]
    }

    # 将四至坐标添加到字典中
    bboxes_dict[key_value] = bbox

# 打印字典
for key, bbox in bboxes_dict.items():
    print(f"Key: {key}, Bounding Box: {bbox}")


In [ ]:

# 定义开始和结束日期
today = datetime.now()
end_date = today.replace(day=1) - timedelta(days=1) if today.day != 1 else today
start_date = end_date - timedelta(days=3*365)  # 三年前

# 修改后的计算月平均降水量函数
def calculate_monthly_average(bbox, year, month):
    start = datetime(year, month, 1)
    end = datetime(year, month+1, 1) if month < 12 else datetime(year+1, 1, 1)
    
    # 创建ee.Geometry.Rectangle
    geometry = ee.Geometry.Rectangle([bbox['MinX'], bbox['MinY'], bbox['MaxX'], bbox['MaxY']])
    
    # 加载IMERG降雨数据集
    precipitations = ee.ImageCollection('NASA/GPM_L3/IMERG_V06') \
        .filterBounds(geometry) \
        .filterDate(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')) \
        .select('precipitationCal')  # 选择降雨量估计波段

    # 计算平均降雨量
    mean_precipitation = precipitations.mean()

    # 使用reduceRegion方法计算区域内的平均降雨量
    mean_value = mean_precipitation.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=20000  # 设置合适的比例尺
    ).getInfo()

    return mean_value.get('precipitationCal', None)

# 创建一个空的DataFrame来存储结果
df = pd.DataFrame()

# 遍历bboxes_dict中的每个面要素
for feature_id, bbox in bboxes_dict.items():
    print(feature_id)
    monthly_averages = {}
    current_year = start_date.year
    current_month = start_date.month

    # 计算每个月的平均降水量
    while current_year < end_date.year or (current_year == end_date.year and current_month <= end_date.month):
        average = calculate_monthly_average(bbox, current_year, current_month)
        monthly_averages[f"{current_year}-{current_month:02d}"] = average
        current_month += 1
        if current_month > 12:
            current_month = 1
            current_year += 1
    
    # 将结果添加到DataFrame
    rows_to_add = pd.DataFrame([{'FeatureID': feature_id, 'Month': month, 'Average Precipitation': avg} for month, avg in monthly_averages.items()])
    df = pd.concat([df, rows_to_add], ignore_index=True)


In [ ]:
df

In [ ]:
df['Average Precipitation'] = df['Average Precipitation']*30*24
df

In [ ]:
# 使用pivot_table重塑DataFrame
pivot_df = df.pivot_table(
    index=['Month'],
    columns='FeatureID',
    values='Average Precipitation'
).reset_index()

print(pivot_df)


In [ ]:
pivot_df

In [44]:
df.to_excel(r"C:\Users\Runker\Desktop\QN_Average_Precipitation_0.xlsx")
pivot_df.to_excel(r"C:\Users\Runker\Desktop\QN_Average_Precipitation.xlsx")